In [55]:
import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

File data/train.txt is already downloaded.
File data/validation.txt is already downloaded.
File data/test.txt is already downloaded.


In [56]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            
            # Replacing all urls with <URL> token
            # Replacing all users with <USR> token
            
            if len(token) > 3 and token[0:4] == "http":
                token = "<URL>";
            if token[0] == '@':
                token = "<USR>";
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [57]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [59]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



In [60]:
from collections import defaultdict

In [61]:
def build_dict(tokens_or_tags, special_tokens):

    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    for subArr in tokens_or_tags:
        for tag in subArr:
            if not tag in idx2tok:
                idx2tok.append(tag)
          
    for tag in special_tokens:
        if not tag in idx2tok:
            idx2tok.append(tag)
     
    for i in range(len(idx2tok)):
        tok2idx[idx2tok[i]] = i
    
    return tok2idx, idx2tok

In [62]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [63]:
print(len(tag2idx))
print(len(idx2tag))

21
21


In [64]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

In [65]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

In [66]:
import tensorflow as tf
import numpy as np

In [67]:
class BiLSTMModel():
    pass

In [68]:
def declare_placeholders(self):

    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')
  
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 
    
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    
    self.learning_rate_ph = tf.placeholder(dtype=tf.int32)

In [69]:
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

In [90]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(initial_value=initial_embedding_matrix, name='embeddings_matrix', dtype=tf.float32)
    
    forward_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_rnn)
    forward_cell = tf.nn.rnn_cell.DropoutWrapper(forward_cell,  input_keep_prob = self.dropout_ph, output_keep_prob = self.dropout_ph, state_keep_prob = self.dropout_ph)
    backward_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_rnn)
    backward_cell = tf.nn.rnn_cell.DropoutWrapper(backward_cell,  input_keep_prob = self.dropout_ph, output_keep_prob = self.dropout_ph, state_keep_prob = self.dropout_ph)

    embeddings =tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)
    
    (rnn_output_fw, rnn_output_bw), _ = tf.nn.bidirectional_dynamic_rnn(forward_cell, backward_cell, embeddings, sequence_length= self.lengths, dtype=tf.float32)
    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

In [91]:
BiLSTMModel.__build_layers = classmethod(build_layers)

In [92]:
def compute_predictions(self):
    softmax_output = tf.nn.softmax(self.logits)
    
    self.predictions = tf.argmax(softmax_output, axis = -1)

In [93]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [149]:
def compute_loss(self, n_tags, PAD_index):
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth_tags_one_hot,logits= self.logits )
    
    mask = tf.cast(tf.not_equal(loss_tensor, PAD_index), tf.float32)
    self.loss =  tf.reduce_mean(tf.multiply(loss_tensor, mask ))

In [150]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

In [151]:
def perform_optimization(self):
    self.optimizer = tf.train.AdamOptimizer(learning_rate = self.learning_rate_ph)
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # list comprehension might be useful in this case.
    clip_norm = tf.cast(1.0, tf.float32)
    self.grads_and_vars = [ (tf.clip_by_norm(pair[0], clip_norm) , pair[1]) for pair in self.grads_and_vars] 
    
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [152]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [153]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [154]:
BiLSTMModel.__init__ = classmethod(init_model)

In [155]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

In [156]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)

In [157]:
def predict_for_batch(self, session, x_batch, lengths):
    feed_dict = {
                 self.input_batch: x_batch,
                 self.lengths: lengths
      }
    predictions = session.run(self.predictions, feed_dict=feed_dict)
    return predictions

In [158]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

In [159]:
from evaluation import precision_recall_f1

In [160]:
def predict_tags(model, session, token_idxs_batch, lengths):
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

## Experiment

In [161]:
tf.reset_default_graph()

model = BiLSTMModel(vocabulary_size = len(idx2token), n_tags = len(idx2tag),embedding_dim = 200,n_hidden_rnn = 200,PAD_index=token2idx['<PAD>'])
batch_size = 32
n_epochs = 4
learning_rate = 0.005
learning_rate_decay = 0.005
dropout_keep_probability = 0.5

If you got an error *"Tensor conversion requested dtype float64 for Tensor with dtype float32"* in this point, check if there are variables without dtype initialised. Set the value of dtype equals to *tf.float32* for such variables.

Finally, we are ready to run the training!

In [162]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_tokens, train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 73774 phrases; correct: 195.

precision:  0.26%; recall:  4.34%; F1:  0.50

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 8933 phrases; correct: 22.

precision:  0.25%; recall:  4.10%; F1:  0.46

-------------------- Epoch 2 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 73774 phrases; correct: 195.

precision:  0.26%; recall:  4.34%; F1:  0.50

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 8933 phrases; correct: 22.

precision:  0.25%; recall:  4.10%; F1:  0.46

-------------------- Epoch 3 of 4 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 1946 phrases; correct: 48.

precision:  2.47%; recall:  1.07%; F1:  1.49

Validation data evaluation:
processed 12836 tokens with 537 phrases; f

In [165]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_tokens, validation_tags, short_report=False) ######### YOUR CODE HERE #############

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags, short_report=False) ######### YOUR CODE HERE #############

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 1392 phrases; correct: 23.

precision:  1.65%; recall:  0.51%; F1:  0.78

	     company: precision:    1.65%; recall:    0.62%; F1:    0.90; predicted:   243

	    facility: precision:    0.00%; recall:    0.00%; F1:    0.00; predicted:    59

	     geo-loc: precision:    2.46%; recall:    0.80%; F1:    1.21; predicted:   325

	       movie: precision:    0.00%; recall:    0.00%; F1:    0.00; predicted:    14

	 musicartist: precision:    0.00%; recall:    0.00%; F1:    0.00; predicted:    88

	       other: precision:    1.55%; recall:    0.40%; F1:    0.63; predicted:   194

	      person: precision:    9.26%; recall:    0.56%; F1:    1.06; predicted:    54

	     product: precision:    0.89%; recall:    0.94%; F1:    0.91; predicted:   338

	  sportsteam: precision:    0.00%; recall:    0.00%; F1:    0.00; predicted:    25

	      tvshow: precision:    0.00%; recall:    0.0